<a href="https://colab.research.google.com/github/wtsyang/UserIntentPrediction/blob/BERT/BERT/BiLSTM-features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Change youe folder
# Make sure you have a folder BERT under this: YOUR_FOLDER/BERT
%cd '/content/drive/My Drive/UserIntentPrediction'

/content/drive/My Drive/UserIntentPrediction


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers  import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
from sklearn.model_selection  import train_test_split
import pickle
import sklearn
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import smart_cond
from functools import partial
from sklearn.metrics import precision_recall_fscore_support
print('Tensorflow Version:',tf.__version__)

Tensorflow Version: 2.2.0-rc3


In [0]:
NUM_MODELS = 1
BATCH_SIZE = 32
LSTM_UNITS = 64
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 15
MAX_LEN = 1259
N_CHANNELS=768
N_CLASS=12
N_Features=24
INDEX_Features=range(-24,-0,1)

## Loading the dataset


In [0]:
Train=pd.read_csv('data/train_features.csv').reset_index(drop=True)
Valid=pd.read_csv('data/valid_features.csv').reset_index(drop=True)
Test=pd.read_csv('data/test_features.csv').reset_index(drop=True)

In [0]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=BATCH_SIZE, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        x = np.zeros((self.batch_size, self.dim, self.n_channels))
        y = np.zeros((self.batch_size,self.n_classes), dtype=int)

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              x[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
            # Store sample
            # Store class
            y[i,:] = np.array(list_IDs_temp.iloc[i,0:12])
        Y=[]
        for i in range(self.n_classes):
          Y+=[y[:,i].reshape((self.batch_size,))]
        X=[x,np.array(list_IDs_temp.iloc[:,INDEX_Features])]
        return X, Y

In [0]:
training_generator = DataGenerator(Train,'Train')
validation_generator = DataGenerator(Valid,'Valid')

In [0]:
classWeight_Dict={}
for i in range(N_CLASS):
  ratioTrue=np.sum(Train.iloc[:,i])/len(Train)
  classWeight_Dict['output'+str(i+1)]={0:1+1/((1-ratioTrue)/(ratioTrue)+1),1:1+(1-ratioTrue)/(ratioTrue)/((1-ratioTrue)/(ratioTrue)+1)}

In [0]:
classWeight_Dict['output4'][1]+=1
classWeight_Dict['output4'][0]+=1

In [10]:
classWeight_Dict

{'output1': {0: 1.234499007936508, 1: 1.765500992063492},
 'output10': {0: 1.0336061507936507, 1: 1.9663938492063493},
 'output11': {0: 1.0100446428571428, 1: 1.9899553571428572},
 'output12': {0: 1.001860119047619, 1: 1.998139880952381},
 'output2': {0: 1.0603918650793651, 1: 1.9396081349206349},
 'output3': {0: 1.0744047619047619, 1: 1.9255952380952381},
 'output4': {0: 2.247147817460317, 1: 2.752852182539683},
 'output5': {0: 1.0875496031746033, 1: 1.9124503968253967},
 'output6': {0: 1.1071428571428572, 1: 1.8928571428571428},
 'output7': {0: 1.3979414682539684, 1: 1.6020585317460316},
 'output8': {0: 1.1070188492063493, 1: 1.8929811507936507},
 'output9': {0: 1.0767609126984128, 1: 1.9232390873015874}}

## Build the model


In [0]:
def binary_crossentropy(y_true, y_pred, weights,from_logits=False,label_smoothing=0):

    y_pred = ops.convert_to_tensor(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    label_smoothing = ops.convert_to_tensor(label_smoothing, dtype=K.floatx())
    def _smooth_labels():
      return y_true * (1.0 - label_smoothing) + 0.5 * label_smoothing
    y_true = smart_cond.smart_cond(label_smoothing,_smooth_labels, lambda: y_true)
    
    mask0 = tf.subtract(tf.constant(1.0, dtype=K.floatx()),y_true)
    mask0=tf.math.scalar_mul(tf.constant(weights[0], dtype=K.floatx()),mask0)
    mask1 =tf.math.scalar_mul(tf.constant(weights[1], dtype=K.floatx()),y_true)
    mask=tf.math.add(mask0,mask1)

    return K.mean(tf.math.multiply(K.binary_crossentropy(y_true, y_pred, from_logits=from_logits),mask), axis=-1)

In [0]:
def build_model():
    inputs1 = Input(shape=(MAX_LEN,N_CHANNELS))
    inputs2 = Input(shape=(N_Features,))

    x = SpatialDropout1D(0.2)(inputs1)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
        inputs2,
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS+24, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS+24, activation='relu')(hidden)])
    RESULT=[]
    for i in range(N_CLASS):
      RESULT+=[Dense(1, activation='sigmoid',name='output'+str(i+1))(hidden)]
    LOSS={}
    for i in  range(N_CLASS):
      LOSS['output'+str(i+1)]=partial(binary_crossentropy, weights=classWeight_Dict['output'+str(i+1)])
      LOSS['output'+str(i+1)].__name__ = 'loss'+str(i+1)

    inputs={'inputs1':inputs1,'inputs2':inputs2}
    model = Model(inputs=inputs, outputs=RESULT)
    model.compile(loss=LOSS, optimizer='adam',metrics=[tf.keras.metrics.binary_accuracy])

    return model

In [13]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1259, 768)]  0                                            
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 1259, 768)    0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1259, 128)    426496      spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1259, 128)    98816       bidirectional[0][0]              
______________________________________________________________________________________________

In [0]:
callback=ReduceLROnPlateau(patience=1,min_lr=0.00001,factor=0.3)
Name='BERT/LSTM_addFeatures.h5'
checkpointer = ModelCheckpoint(filepath=Name, verbose=1, save_best_only=True)

In [15]:
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpointer,callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Faile to load the data: BERT/vector/Train_181754_19353.npy
Epoch 1/15
252/252 [==============================] - ETA: 0s - loss: 7.2813 - output1_loss: 0.5729 - output2_loss: 0.5718 - output3_loss: 0.4914 - output4_loss: 1.9815 - output5_loss: 0.4884 - output6_loss: 0.8446 - output7_loss: 0.6572 - output8_loss: 0.5088 - output9_loss: 0.4499 - output10_loss: 0.1720 - output11_loss: 0.3232 - output12_loss: 0.2197 - output1_binary_accuracy: 0.8552 - output2_binary_accuracy: 0.9253 - output3_binary_accuracy: 0.9159 - output4_binary_accuracy: 0.7124 - output5_binary_accuracy: 0.9028 - output6_binary_accuracy: 0.8571 - output7_binary_accuracy: 0.8053 - output8_binary_accuracy: 0.8785 - output9_binary_accuracy: 0.9134 - output10_binary_accuracy: 0.9685 - output11_binary_accuracy: 0.9720 - output12_binary_accuracy: 0.9833
Epoch 00001: val_loss improved from inf to 4.72480, saving model to BERT/LSTM_addFeatures.h5
252/2

## Prediction




In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1259, 768)]  0                                            
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 1259, 768)    0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1259, 128)    426496      spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1259, 128)    98816       bidirectional[0][0]              
______________________________________________________________________________________________

In [0]:
model.load_weights(Name)

In [0]:
class testDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=25, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
         # Initialization
        x = np.zeros((self.batch_size, self.dim, self.n_channels))

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              x[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')

          
        X=[x,np.array(list_IDs_temp.iloc[:,INDEX_Features])]
        return X

In [0]:
test_generator=testDataGenerator(Test,'Test')

In [20]:
prediction = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


In [0]:
Prediction=np.array(prediction)

In [0]:
y_true=np.array(Test.iloc[:,0:N_CLASS])
y_pred=Prediction

In [0]:
def hamming_score(y_true, y_pred, toggle_output=False):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_pred.shape[1]):
        set_true = set( np.where(y_true[i,:])[0])
        set_pred = set( np.where(y_pred[:,i,0]>=0.5)[0])
        if toggle_output:
            print('set_true: {0}'.format([id2label[id] for id in set_true]), 'set_pred: {0}'.format([id2label[id] for id in set_pred]))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [24]:
hamming_score(y_true, y_pred, toggle_output=False)

0.6816936936936936

In [0]:
   
def f1(y_true, y_pred):
    correct_preds, total_correct, total_preds = 0., 0., 0.
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i,:])[0])
        set_pred = set( np.where(y_pred[:,i,0]>=0.5)[0])
        
        correct_preds += len(set_true & set_pred)
        total_preds += len(set_pred)
        total_correct += len(set_true)

    p = correct_preds / total_preds if correct_preds > 0 else 0
    r = correct_preds / total_correct if correct_preds > 0 else 0
    f1 = 2 * p * r / (p + r) if correct_preds > 0 else 0
    return p, r, f1

In [26]:
f1(y_true, y_pred)

(0.768624014022787, 0.6525297619047619, 0.7058350100603622)